In [1]:
#%%
from model import Generator, Discriminator, DomainClassifier
from utils import audio_process
import IPython.display as ipd
import tensorflow as tf
import numpy as np
import os

In [4]:
#%%
'''
Usage:
  Interactive web page via jupyter
'''
ckpt_path = './ckpt'

G = Generator()
dummy_wav = np.zeros((1, 36, 512, 1), dtype=np.float32)
dummy_speaker_onehot = np.zeros((1, 2), dtype=np.float32)
G(dummy_wav, dummy_speaker_onehot)
restored_ckpt = tf.train.Checkpoint(G=G)
latest_ckpt = tf.train.latest_checkpoint(ckpt_path)
restored_ckpt.restore(latest_ckpt).assert_existing_objects_matched()

In [5]:
#%%
result_dir = 'result'
input_wav = '../dataset/speakers/p1/p1_002.wav'
target_speaker = 1
speaker_counts = 2

mag, mel = audio_process.wav2spectrogram(input_wav)
mel = mel.astype(np.float32)
mel = mel[np.newaxis, :, :, np.newaxis]
speaker = np.eye(speaker_counts, dtype=np.float32)[target_speaker]
speaker = speaker[np.newaxis, :]
print(speaker)

converted = G(mel, speaker)
converted_np = converted.numpy()[0,:,:,0]

converted_wav = audio_process.melspectrogram2wav(converted_np)

filename = input_wav.split('/')[-1].split('.')[0]
filename = filename + '_to_speaker%d.wav' % (target_speaker)
filepath = os.path.join(ckpt_path, result_dir, filename)
os.makedirs(os.path.dirname(filepath), exist_ok=True)
abs_filepath = audio_process.write_wav(converted_wav, filepath)
ipd.Audio(abs_filepath)

[[0. 1.]]
